<a href="https://colab.research.google.com/github/ali-kmirzaei/NLP/blob/main/05-emojify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# libs

In [ ]:
!pip install emoji

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import keras

import csv
import emoji

%matplotlib inline

# read data

In [ ]:
!unzip emoji.zip

In [4]:
def read_csv(filename):
  phrase = []
  emoji = []
  with open (filename) as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
      phrase.append(row[0])
      emoji.append(row[1])
    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)
    return X, Y

In [5]:
X_train, Y_train = read_csv('train.csv')
X_test, Y_test = read_csv('test.csv')

In [6]:
maxlen = len(max(X_train, key=len).split())
maxlen

10

# label to emoji

In [41]:
emoji_dict = {
    "0" : ":red_heart:",
    "1" : ":baseball:",
    "2" : "\U0001F604",
    "3" : "\U0001F61E",
    "4" : ":fork_and_knife:"
}

def label_to_emoji(label):
  return emoji.emojize( emoji_dict[ str(label) ] )

In [42]:
for i in range(10):
  print(X_train[i], label_to_emoji(Y_train[i]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾


# read glove file as pretrained embeddings

In [74]:
def read_glove_vecs(glove_file):
  with open(glove_file, encoding="utf8") as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      line = line.strip().split()
      curr_word = line[0]
      words.add(curr_word)
      
      if '-1.377.0693' in line:
        c = 0
        for t in line:
          if t == '-1.377.0693':
            line[c] = '-1.377'
          c += 1
      word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float)

    i = 1
    words_to_index = {}
    index_to_words = {}
    for w in sorted(words):
      words_to_index[w] = i
      index_to_words[i] = w
      i += 1
    return words_to_index, index_to_words, word_to_vec_map

In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [86]:
# Test
word = "ali"
index = 113317
word_to_index[word], index_to_word[index], word_to_vec_map[word]

(51314,
 'cucumber',
 array([-0.71587 ,  0.7874  ,  0.71305 , -0.089955,  1.366   , -1.3149  ,
         0.7309  ,  0.79725 ,  0.47211 ,  0.53347 ,  0.37542 , -0.10256 ,
        -1.0003  , -0.31226 ,  0.26217 ,  0.92426 ,  0.43014 , -0.015593,
         0.4149  ,  0.88286 ,  0.10869 ,  0.95213 ,  1.1807  ,  0.06445 ,
        -0.05814 , -1.797   , -0.18432 , -0.41754 , -0.73625 ,  1.1607  ,
         1.5932  , -0.70268 , -0.61621 ,  0.47118 ,  0.95046 ,  0.35206 ,
         0.6072  ,  0.59339 , -0.47091 ,  1.4916  ,  0.27146 ,  1.8252  ,
        -1.2073  , -0.80058 ,  0.52558 , -0.33346 , -1.4102  , -0.21514 ,
         0.12945 , -0.69603 ]))